In [6]:
# -*- coding: utf-8 -*-
import re
# , AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
#from transformers import pipeline
from multiprocessing import Pool
from deeppavlov import configs, build_model
#import yake
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.chunk import conlltags2tree
from nltk.tree import Tree
from nltk import pos_tag
import math
from nltk.corpus import stopwords
#from enchant.checker import SpellChecker
#from transformers import T5Tokenizer, T5ForConditionalGeneration
#from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import pandas as pd
#from sentence_transformers import SentenceTransformer
from scipy.spatial import distance
from tqdm import tqdm
import numpy as np
import json
import argparse
import logging
import random
import torch
from utils import evaluate_scores
import spacy
import os
import sys
sys.path.insert(0, os.path.abspath('..'))


In [9]:
train_df = pd.read_csv('../data/semeval-2022_task8_eval_data_202201_paired_articles_articles.csv')
train_df.head(40)

,pair_id,title1,text1,title2,text2,meta_keywords1,meta_keywords2,entities1,entities2,topic1,topic2,top_images1,top_images2,keywords1,keywords2,tags1,tags2
0,1484189203_1484121193,Police: 2 men stole tools from Lowe’s in Davie,"DAVIE, FLA. (WSVN) - Police need help catching...",No-swim advisory lifted for Deerfield Beach Pier,"DEERFIELD BEACH, FLA. (WSVN) - A no-swim advis...",[''],[''],"[('2', 'CARDINAL'), ('Lowe ’ s', 'ORG'), ('Dav...","[('Deerfield Beach Pier', 'FAC'), ('FLA', 'GPE...",NaN,NaN,True,True,[],[],"['More Trending Stories', 'Trending']","['More Trending Stories', 'Trending']"
1,1484011097_1484011106,"Open database leaked 179GB in customer, US gov...",Govt officials confirm Trump can block US comp...,Best Western’s Massive Data Leak: 179GB Amazon...,The latest huge unsecured cloud storage find i...,[''],[''],"[('179GB', 'CARDINAL'), ('US', 'GPE'), ('Trump...","[('179GB', 'CARDINAL'), ('Autoclerk', 'ORG'), ...",NaN,NaN,True,True,[],[],"['Innovation', 'AI', 'Microsoft', 'Security TV...","['Data breach', 'Best Western', 'security', 'I..."
2,1484039488_1484261803,Ducks are own worst enemies in sloppy loss in ...,"Ducks defenseman Erik Gudbranson, left, knocks...",Woody Guthrie's 1943 New Year's Resolutions ar...,Woody Guthrie's 1943 New Year's Resolutions ar...,[''],[''],"[('Ducks', 'ORG'), ('Las Vegas Ducks', 'ORG'),...","[(""Woody Guthrie's"", 'PERSON'), ('1943', 'DATE...",NaN,NaN,True,True,[],[],['anaheim-ducks'],"['resolutions', 'beat fascism', ""new year's re..."
3,1484332324_1484796748,Another Bengal vs Centre tussle? Govt rejects ...,The West Bengal government’s proposal was reje...,'Congress Rejected 7 Times': BJP's Reminder as...,Mumbai: The NCP and Shiv Sena on Thursday targ...,"['republic day', 'west bengal tableau', 'benga...","['BJP', 'congress', 'Mamata Banerjee', 'NCP', ...","[('Bengal', 'GPE'), ('Republic Day', 'DATE'), ...","[(""7 Times'"", 'WORK_OF_ART'), (""BJP's"", 'ORG')...",NaN,NaN,True,True,[],[],[],[]
4,1484012256_1484419682,Bars and clubs you loved and lost this decade ...,The video will start in 8 Cancel\n\nSign up to...,Top 20 films of the 2010s,"Jacksonville, FL - I'm not sure how we'll look...","['Birmingham City Centre', 'Digbeth', 'Things ...","['organizing', 'activism', 'socialism', ""Peopl...","[('this decade', 'DATE'), ('Birmingham', 'GPE'...","[('20', 'CARDINAL'), ('the 2010s', 'DATE'), ('...",NaN,NaN,True,True,[],[],[],['Movies']
5,1484188974_1484299664,How to free up iCloud storage space,"Lex uses a MacBook Pro, an iPhone 4S, a third-...",Leading and managing organisational change | T...,\n\nEvery change begins with a leadership deci...,[''],[''],"[('iCloud', 'ORG'), ('Lex', 'ORG'), ('a MacBoo...","[('The Guardian Nigeria News', 'ORG'), ('Niger...",NaN,NaN,True,True,[],[],"['Apple', 'iOS', 'iCloud']",['Akindotun Merino']
6,1484703270_1484167784,Op Ed: Losing Sheila’s Smile,Get a daily rundown of the top stories on Urba...,Gov. Tony Evers says relationship with Republi...,He said he's hopeful that this coming year wil...,[''],"['', 'tony evers', 'politics', 'institutes', '...","[('Ed', 'PERSON'), ('Sheila', 'PERSON'), ('Urb...","[('Tony Evers', 'PERSON'), ('Republican', 'NOR...",NaN,NaN,True,True,[],[],[],[]
7,1484012638_1483801741,President: The Azerbaijani government is alway...,"Baku, January 1, AZERTAC\n\n""Problems of the I...",Azerbaijan’s president: 2019 will go down in h...,"Baku, Azerbaijan, Jan. 1\n\nTrend:\n\n""2019 wi...","['President Ilham Aliyev', 'congratulate', 're...","[""Azerbaijan's news"", ""Georgia's news"", ""Kazak...","[('Azerbaijani', 'NORP'), ('Baku', 'GPE'), ('J...","[('Azerbaijan', 'GPE'), ('2019', 'DATE'), ('a ...",NaN,NaN,True,True,[],[],[],['Ilham Aliyev']
8,1483805592_1484139101,Meteor shower visible in UAE skies on first Fr...,The Quadrantids can shower around 50 to 100 me...,UAE Peeps Will Have The Best View In The House...,If you haven’t made any long drive or desert p...,[''],[''],"[('UAE', 'GPE'), ('first Friday of 2020', 'DAT...","[('UAE', 'GPE')

In [6]:
data_path = '../data/train_dir/batch2'

In [22]:
def get_text(pair_id):
    path = os.path.join(data_path, pair_id[-2:], pair_id + ".json")

    article_text = ''
    if os.path.exists(path):
        with open(path, 'r') as f:
            data = json.load(f)
        article_text = data['text']
    else:
        print(path)
        
    article_text = re.sub(r"\n+", ' ', article_text)
    article_text = re.sub(r"\s+", ' ', article_text)
    return article_text


In [23]:

train_df = pd.read_csv('../data/train_split_batch2.csv')
test_df = pd.read_csv('../data/test_split_batch2.csv')

test_df['text1'] = test_df['pair_id'].apply(
    lambda pair: get_text(pair.split("_")[0]))
test_df['text2'] = test_df['pair_id'].apply(
    lambda pair: get_text(pair.split("_")[1]))

print(test_df.head())


../data/train_dir/batch2/64/1559167664.json
../data/train_dir/batch2/97/1561076497.json
../data/train_dir/batch2/80/1484184080.json
../data/train_dir/batch2/45/1522138345.json
../data/train_dir/batch2/30/1587715330.json
../data/train_dir/batch2/67/1484037967.json
../data/train_dir/batch2/08/1517994808.json
../data/train_dir/batch2/21/1550144921.json
../data/train_dir/batch2/91/1485167191.json
../data/train_dir/batch2/06/1484039906.json
../data/train_dir/batch2/94/1608646894.json
../data/train_dir/batch2/59/1484011259.json
../data/train_dir/batch2/00/1484040300.json
../data/train_dir/batch2/54/1484038154.json
../data/train_dir/batch2/29/1612074329.json
../data/train_dir/batch2/64/1484010964.json
../data/train_dir/batch2/57/1483806257.json
../data/train_dir/batch2/06/1558196706.json
../data/train_dir/batch2/74/1647871474.json
../data/train_dir/batch2/92/1605872292.json
../data/train_dir/batch2/12/1484012212.json
../data/train_dir/batch2/14/1530033014.json
../data/train_dir/batch2/38/1484

In [24]:
for _, item in test_df.iterrows():
    print('[', item.url1_lang, item.url2_lang, ']', '-', item.Overall)
    sentences = sent_tokenize(item.text1)
    if len(sentences) > 1:
        print('1: ', sentences[0][:90] + '....')
    else:
        print('1: --', item.text1, '--')
        #import pdb;pdb.set_trace()
    sentences = sent_tokenize(item.text2)
    if len(sentences) > 1:
        print('2: ', sentences[0][:90] + '....')
    else:
        print('2: --', item.text2, '--')

[ fr fr ] - 3.0
1:  Le cap du million de ventes a été dépassé l'an dernier.....
2:  Stocks réduits ou distribués au "compte-gouttes", les soignants français manquent toujours....
[ fr fr ] - 4.0
1:  SPORTS – La majorité des activités sportives de la planète ont été suspendues en raison de....
2:  Cette nuit, la Chambre des communes a adopté le projet de loi de 82 G$ d’aide d’urgence au....
[ fr fr ] - 2.0
1:  Deux Matanais, Hélène Gagnon et Clarence Bouffard, font partie de la centaine de Canadiens....
2:  Sarah Mahu est coincée au Pérou.....
[ fr fr ] - 1.0
1:  Atteint par le covid-19, l’artiste camerounais Manu Dibango est mort mardi matin des suite....
2:  L'Afrique pleure Manu Dibango, victime du coronavirus (Autre presse) [ALomé] - 25/03/2020 ....
[ fr fr ] - 3.0
1:  C’est dans le cadre de la crise générée par la COVID-19 que la MRC de Marguerite-D’Youvill....
2:  Pour la version anglais, cliquez ici.....
[ fr fr ] - 3.0
1:  L'Américain Robert Levinson, agent du FBI à la retraite 

1:  18:42 Felipe Guevara se refirió al multitudinario festejo que se realizó anoche en el luga....
2:  10:08 Durante las fiestas miles de personas salieron de Santiago, y se espera un masivo re....
[ tr tr ] - 3.0
1:  Mamak'ta koronavirüs önlemleri Mamak Belediye Başkanı Murat Köse, Kovid-19 nedeniyle istih....
2:  08.01.2020 12:30 | Son Güncelleme: 08.01.2020 12:30 Mamak Belediyesince ilçe sınırları içi....
[ de de ] - 1.0
1:  Mindestens neun Menschen in der Schweiz sind am Corona-Virus erkrankt.....
2:  Neun Corona-Fälle – und die Zahl der Infektionen dürfte weiter steigen Am Donnerstag ist d....
[ en en ] - 4.0
1:  From the private funicular that takes you to model Cheryl Tiegs’s Palm Springs hideaway to....
2:  Look ahead: Twenty things to watch out for in 2020 Twenty things to track at the start of ....
[ de en ] - 3.0
1:  Mehrere junge Klimaaktivisten haben in Davos eine stärkere Mitsprache ihrer Generation bei....
2:  Another World Economic Forum is kicking off in Davos, Switzer

1:  News-Ticker für den Freistaat + © dpa / Tobias Hase Corona/Bayern: Söder knöpft sich Skept....
2:  A day after he was part of Chancellor Angela Merkel’s Corona conference with Germany’s fed....
[ de de ] - 3.0
1:  Potsdam - Der Brand in der Asta-Nielsen-Straße am Samstag in Drewitz wurde nach Angaben de....
2:  Potsdam - Am Dienstagabend um kurz vor 22 Uhr wurde es laut an der Haltestelle Robert-Babe....
[ es es ] - 1.0
1:  Oaxaca.- Una mujer murió por balas perdidas y dos más resultaron heridas durante los feste....
2: -- Escuchar Nota Los festejos de Año Nuevo en algunas regiones de Oaxaca se caracterizan por ser muy particulares, y es queel inicio del año.Sin embargo, estas prácticas, además de dos lesionados, luego de que las balas cayeron sobre algunas casas y calles de Salina Cruz y Pinotepa Nacional.Durante la madrugada de Año Nuevo, una mujer identificada como Gabriela, en Salina Cruz.Los hechos se registraron en la calle Periférico en el Barrio Juárez de ese municipio del 

[ en en ] - 4.0
1:  This is the 216th world-famous FOTM Caption Contest, the first contest of 2020!....
2:  Share Tweet Email address: Leave this field empty if you're human: Welcome to the first ad....
[ de de ] - 4.0
1:  Wegen Coronakrise: Schweiz verzichtet auf Teilnahme an Europäischer Impfwoche Am Montag ha....
2:  Betrüger, Abzocker und Cyber-Kriminelle nutzen die Ängste der Schweizer Bevölkerung aus, u....
[ de en ] - 4.0
1:  Bern.....
2:  If youâre dreaming of escaping to Oz, the present day property scene might persuade you ....
[ ar ar ] - 3.0
1:  كشفت إحدى صديقات الفنانة دنيا عبد العزيز عن موعد إقامة عزاء والدتها.....
2:  نشرت الفنانة دنيا عبد العزيز عبر حسابها الشخصي على موقع الصور الشهير "انستجرام" صورة جديدة....
[ de de ] - 2.0
1:  Sidney Hoffmann, Autotuner aus Dortmund, startet mit "Sidneys Welt" (DMAX) seine eigene TV....
2:  Große, aber erfreuliche Überraschung für die Fans von Sidney Hoffmann.....
[ es es ] - 1.0
1:  Rooney ya es oficialmente jugador del Derby Coun